In [78]:
import pandas as pd
import sqlite3
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [79]:
with sqlite3.connect("../music.db") as conn:
    df = pd.read_sql_query("SELECT * FROM MUSIC", conn)
# df

In [80]:
df

,id,song_name,artist_name,genre,year_released,country_released,image_cover_link,youtube_link,styles
0,1,"Audio, Video, Disco.",Justice (3),"Electronic, Rock, Pop",2011,France,https://i.discogs.com/Arx-Odgpe_7A4wnF2Sf4PM1U...,https://www.youtube.com/watch?v=4snFQ_0evXA,"House, Electro, Rock & Roll"
1,2,うたかたの日々= Utakata No Hibi,Mariah (3),Electronic,2015,US,https://i.discogs.com/JhM04o_CN6yqVwk8R_KJTKSL...,https://www.youtube.com/watch?v=xCW2p4HYPJg,Synth-pop
2,3,Mit Pfefferminz Bin Ich Dein Prinz,Marius Müller-Westernhagen,Rock,2000,Germany,https://i.discogs.com/pUumG2YKg5iV9IxLdRz_vMg2...,https://www.youtube.com/watch?v=pVoAEV5Vvs8,"Blues Rock, Rock & Roll, Acoustic"
3,4,One-Nine-Nine-Nine / Like They Used To Say,Common,Hip Hop,1999,US,https://i.discogs.com/cF2JYcS1n3q3e6I5cuiEK_4i...,https://www.youtube.com/watch?v=5OfWXGx_0-4,
4,5,Monoliths & Dimensions,Sunn O))),Rock,2009,US,https://i.discogs.com/2ZIXCBzXaK-piVco0Ld72tHH...,https://www.youtube.com/watch?v=T5P7bo5hKgQ,"Doom Metal, Experimental, Drone"
...,...,...,...,...,...,...,...,...,...
1059,1060,Son Of A Bitch,Menace II Society,Electronic,2002,Netherlands,https://i.discogs.com/wPvHaOBzRNwMb86I0_NEvWEW...,https://www.youtube.com/watch?v=OnO4pX3injs,"Hardcore, Gabber"
1060,1061,Surrealistic Pillow,Jefferson Airplane,Rock,2000,US,https://i.discogs.com/sveaJXQBRGLQU2sLYEnKslWD...,https://www.youtube.com/watch?v=cEY9deRvV2w,"Pop Rock, Psychedelic Rock"
1061,1062,Sonik Source EP,Force Mass Motion,Electronic,1993,UK,https://i.discogs.com/Pqbqhy14TL1q6Vglwn93_cli...,https://www.youtube.com/watch?v=-vL6RW9jiLo,"Breakbeat, Acid, Techno"
1062,1063,Shades Of Deep Purple,Deep Purple,Rock,2000,US,https://i.discogs.com/B3shQW72kyg0Z3HAHWXh5XQH...,https://www.youtube.com/watch?v=J3sqPIGGwHg,Psychedelic Rock


In [81]:
df_encoded = df.copy()

df_encoded['artist_name'] = df_encoded['artist_name'].apply(lambda x: re.sub(r'\s*\(\d+\)$', '', x))

one_hot = pd.get_dummies(df_encoded['artist_name'])
df_encoded = df.join(one_hot)

one_hot = pd.get_dummies(df['year_released'])
df_encoded = df.join(one_hot)

df_encoded= df_encoded.drop(columns=['artist_name', 'year_released'])

df_encoded['styles'] = df_encoded['styles'].str.split(', ')
styles_dummies = df_encoded['styles'].apply(lambda x: pd.Series([1]*len(x), index=x)).fillna(0, downcast='infer')
df_encoded = pd.concat([df_encoded.drop('styles', axis=1), styles_dummies], axis=1)

df_encoded['genre'] = df_encoded['genre'].str.split(', ')
styles_dummies = df_encoded['genre'].apply(lambda x: pd.Series([1]*len(x), index=x)).fillna(0, downcast='infer')
df_encoded = pd.concat([df_encoded.drop('genre', axis=1), styles_dummies], axis=1)

df_encoded['country_released'] = df_encoded['country_released'].str.split('& ')
styles_dummies = df_encoded['country_released'].apply(lambda x: pd.Series([1]*len(x), index=x)).fillna(0, downcast='infer')
df_encoded = pd.concat([df_encoded.drop('country_released', axis=1), styles_dummies], axis=1)
df_encoded = df_encoded.drop(columns=['', 'song_name', 'image_cover_link', 'youtube_link'])
df_encoded

,id,1957,1958,1959,1961,1962,1963,1964,1965,1966,...,Unknown,Mexico,Norway,Uruguay,Denmark,Australasia,France,"USA, Canada",Luxembourg,USSR
0,1,False,False,False,False,False,False,False,False,False,...,0,0,0,0,0,0,0,0,0,0
1,2,False,False,False,False,False,False,False,False,False,...,0,0,0,0,0,0,0,0,0,0
2,3,False,False,False,False,False,False,False,False,False,...,0,0,0,0,0,0,0,0,0,0
3,4,False,False,False,False,False,False,False,False,False,...,0,0,0,0,0,0,0,0,0,0
4,5,False,False,False,False,False,False,False,False,False,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1059,1060,False,False,False,False,False,False,False,False,False,...,0,0,0,0,0,0,0,0,0,0
1060,1061,False,False,False,False,False,False,False,False,False,...,0,0,0,0,0,0,0,0,0,0
1061,1062,False,False,False,False,False,False,False,False,False,...,0,0,0,0,0,0,0,0,0,0
1062,1063,False,False,False,False,False,False,False,False,False,...,0,0,0,0,0,0,0,0,0,0


In [82]:
for col in df_encoded.columns:
    print(col)

id
1957
1958
1959
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
House
Electro
Rock & Roll
Synth-pop
Blues Rock
Acoustic
Doom Metal
Experimental
Drone
Deep House
Acid House
Heavy Metal
Hard Rock
Soft Rock
Country
Folk
Holiday
Classic Rock
Pop Rock
Alternative Rock
Garage Rock
Punk
Black Metal
Avantgarde
Gospel
Post Rock
Bubblegum
Vocal
Easy Listening
Soul
Hardcore
Oi
Garage House
Indie Rock
Prog Rock
Folk Rock
Disco
Funk
Eurodance
Euro House
Glam
Art Rock
Trip Hop
Abstract
Downtempo
Techno
Ambient
Instrumental
Tech House
Minimal
Breakbeat
Acid
Drum n Bass
Arena Rock
Progressive House
Breaks
Progressive Trance
Post-Punk
New Beat
Thrash
Comedy
Parody
Jungle
Ballad
Dub
Dub Techno
Gabber
Hardstyle
Minneapolis Sound
Noise
Hard Tranc

In [83]:
data = df_encoded.drop('id', axis=1)

In [87]:
# assuming data is your one-hot encoded DataFrame
similarity_matrix = cosine_similarity(data)


In [107]:
def recommend_items(user_id, similarity_matrix, items, top_k):
    """
    Recommend top K items for a given user.
    
    Parameters:
    - user_index: Index of the user for which to generate recommendations
    - similarity_matrix: Precomputed item similarity matrix
    - items: List of item names/titles
    - top_k: Number of recommendations to generate
    
    Returns:
    - List of top K recommendations
    """
    user_id += 1
    
    # Get similarity scores for all items with respect to the given user
    similarity_scores = similarity_matrix[user_id]

    # Sort items based on similarity scores
    sorted_indices = np.argsort(similarity_scores)[::-1]

    # Get top-K most similar items
    top_k_indices = sorted_indices[:top_k]

    # Return recommended items
    return [items[i] for i in top_k_indices]


In [115]:
b = 108

In [121]:
items = df['id'].tolist()

a = recommend_items(b, similarity_matrix, items, 5)

In [122]:
a

[110, 454, 879, 314, 770]

In [123]:
row = df.iloc[b]
row

id                                                                109
song_name                        Fat Music Vol. III: Physical Fatness
artist_name                                                   Various
genre                                                            Rock
year_released                                                    1997
country_released                                                   US
image_cover_link    https://i.discogs.com/Z_S2X0HzTo68PVfFnETEB0sy...
youtube_link              https://www.youtube.com/watch?v=HfFXusCR5wk
styles                                                 Hardcore, Punk
Name: 108, dtype: object

In [119]:
row = df.iloc[a[0]]
row

id                                                                111
song_name                                                  Out My Way
artist_name                                              Meat Puppets
genre                                                            Rock
year_released                                                    1986
country_released                                                   US
image_cover_link    https://i.discogs.com/EwBqYyo6Lh-TbtJe8_Og0WHk...
youtube_link              https://www.youtube.com/watch?v=l55R1CVeEHo
styles                             Alternative Rock, Psychedelic Rock
Name: 110, dtype: object

In [120]:
row = df.iloc[a[1]]
row

id                                                                455
song_name                                          Psychometry Vol. 1
artist_name                                                    Akufen
genre                                                      Electronic
year_released                                                    2001
country_released                                              Germany
image_cover_link    https://i.discogs.com/RQ4hnI89s3u33bhKbuW3AKqn...
youtube_link              https://www.youtube.com/watch?v=ECx3Bd12LjU
styles                                            Minimal, Tech House
Name: 454, dtype: object